# insert new values scraped from imdb to collection

In [1]:
import pymongo
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["contelligenz"]
mycol = mydb["movies"]

In [34]:
mycol.update_many({}, {"$set": {"reviews": []}}, upsert=False, array_filters=None)
mycol.update_many({}, {"$set": {"summaries": []}}, upsert=False, array_filters=None)
mycol.update_many({}, {"$set": {"synopsis": ""}}, upsert=False, array_filters=None)
mycol.update_many({}, {"$set": {"cast": []}}, upsert=False, array_filters=None)


In [3]:
df_ = pd.read_csv("/Users/atufashireen/projects/Tamu_Bloomberg/scraped_titles_1.csv")

In [13]:
df_.dropna(inplace=True)

In [19]:
class ScrapeImdb():
    def __init__(self):
        self.headers  = { 'Accept-Language' : "en-US,en;q=0.9,hi;q=0.8,ur;q=0.7,bn;q=0.6",
            'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

    def create_soup_review(self,id):
        url = f"https://www.imdb.com/title/{id}/reviews?ref_=tt_urv"
        content = requests.get(url,headers=self.headers).text
        soup = BeautifulSoup(content, 'html.parser')
        return soup
    
    def create_soup_summs(self,id):
        url = f"https://www.imdb.com/title/{id}/plotsummary?ref_=ttpl_ql_3"
        content = requests.get(url,headers=self.headers).text
        soup = BeautifulSoup(content, 'html.parser')
        return soup
    
    def create_soup_cast(self,id):
        url = f"https://www.imdb.com/title/{id}/fullcredits?ref_=tt_cl_sm"
        content = requests.get(url,headers=self.headers).text
        soup = BeautifulSoup(content, 'html.parser')
        return soup

    def get_reviews(self,id):
        reviews = []
        soup  = self.create_soup_review(id)
        review = soup.find("div",id="main").find("section",class_="article").find("div",class_="lister-list").find_all("div",class_='content')
        for i in review:
            reviews.append(i.text)
        return reviews
        
    def get_synopsis_summ(self,id):
        summs = []
        soup  = self.create_soup_summs(id)
        summ = soup.find("div",id='main').find("ul",id="plot-summaries-content").find_all("li")
        for i in summ:
            summs.append(i.text)
        syn = soup.find("ul",id='plot-synopsis-content').text

        return summs, syn
    
    def get_cast_crew(self,id):
        soup = self.create_soup_cast(id)
        casts_ = []
        try:
            casts = soup.find("table", class_="cast_list").find_all("td",class_="primary_photo")
            casts = [i.find_next("td").text for i in casts]
            casts = [re.sub("/n","",i).strip() for i in casts]
            for i in casts:
                casts_.append(i)
            return casts_[:4]
        except Exception as e:
            print("excepriotn",e)
            return casts_


In [35]:
scrap = ScrapeImdb()
for i in range(0, len(df_)):
    print("doing..", i)
    id = df_.iloc[i]['ID']
    revs  = scrap.get_reviews(id)
    mycol.update_one({'id':id}, {"$push":{'reviews':{'$each':revs}}})
    summs,syn  = scrap.get_synopsis_summ(id)
    mycol.update_one({'id':id}, {"$push":{'summaries':{'$each':summs}}})
    mycol.update_one({'id':id}, {"$set":{'synopsis':syn}})
    casts = scrap.get_cast_crew(id)
    print(casts)
    mycol.update_one({'id':id}, {"$push":{'cast':{'$each':casts}}})

doing.. 0
['Aamir Khan', 'Madhavan', 'Sharman Joshi', 'Kareena Kapoor']
doing.. 1
['Aamir Khan', 'Rajinder Sharma Nanu', 'Anushka Sharma', 'Sushant Singh Rajput']
doing.. 2
['Aamir Khan', 'Fatima Sana Shaikh', 'Sanya Malhotra', 'Sakshi Tanwar']
doing.. 3
['Aamir Khan', 'Fatima Sana Shaikh', 'Sanya Malhotra', 'Sakshi Tanwar']
doing.. 4
['Aamir Khan', 'Gracy Singh', 'Rachel Shelley', 'Paul Blackthorne']
doing.. 5
['Shah Rukh Khan', 'Kajol', 'Katie Amanda Keane', 'Kenton Duty']
doing.. 6
['Shah Rukh Khan', 'Gayatri Joshi', 'Kishori Ballal', 'Smith Seth']
doing.. 7
['Manoj Bajpayee', 'Richa Chadha', 'Nawazuddin Siddiqui', 'Tigmanshu Dhulia']
doing.. 8
['Manoj Bajpayee', 'Richa Chadha', 'Nawazuddin Siddiqui', 'Tigmanshu Dhulia']
doing.. 9
['Ranbir Kapoor', 'Priyanka Chopra Jonas', "Ileana D'Cruz", 'Saurabh Shukla']
doing.. 10
['Ranbir Kapoor', 'Priyanka Chopra Jonas', "Ileana D'Cruz", 'Saurabh Shukla']
doing.. 11
['Sanjay Dutt', 'Arshad Warsi', 'Gracy Singh', 'Boman Irani']
doing.. 12
['Sal